In [2]:
from __future__ import print_function, division
from io import open
import string
import re
import random
import pandas as pd
import os
import sys
import argparse
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle

In [10]:
# reparse HF data and separate demographics
dc_l=[15566,109,84,267,62,144,379,149,61,231,13202,154,345,83,63,85,4263,184,1036]
dc_a_l=[109,267,144,379,149,61,231,13202,345,83,1036]
dc_g_r_l=[15566,84,62,154,63,85,4263,184]

for h in [143]: #[104 ,118 ,131 ,154 ,212 ,225 ,384 ,85 ,14 ,143]:
    t_file= '/data/projects/cerner_dev/cl2_hosp_pdata/h50_cl2_all_hosp_'+ str(h)
    t_seqfile= t_file +'.visits.'
    t_labfile= t_file +'.labels.'
    t_timefile= t_file +'.days.'
    t_ptfile= t_file +'.pts.'
    
    for s in ['test','train','valid']:
        fset=[]
        t_seqfile_s=t_seqfile+s
        t_labfile_s=t_labfile+s
        t_timefile_s= t_timefile +s
        t_ptfile_s= t_ptfile +s
        set_x = pickle.load(open(t_seqfile_s, 'rb'),encoding= 'bytes')
        set_y = pickle.load(open(t_labfile_s, 'rb'),encoding= 'bytes')
        set_t = pickle.load(open(t_timefile_s, 'rb'),encoding= 'bytes')
        set_p = pickle.load(open(t_ptfile_s, 'rb'),encoding= 'bytes')
        for pt_idx in range(len(set_y)):
            pt_sk= set_p[pt_idx]
            pt_lbl= set_y[pt_idx]
            pt_vis= set_x[pt_idx]
            pt_td= set_t[pt_idx]
            d_gr=[]
            n_seq=[]
            d_a_v=[]
            for v in range(len(pt_vis)):
                    nv=[]
                    nv.append([pt_td[v]])
                    for c in pt_vis[v]:
                        if (c in dc_g_r_l) and (c not in d_gr): 
                                d_gr.append(c) 
                        elif c in dc_a_l:
                                d_a_v.append(c)                  
                        else: 
                            nv.append(c)                   
                   #n_seq.append([d_a_v,nv]) ### age group is a visit sublist
                    n_seq.append(nv)
            d_gr.append(d_a_v[0]) 
            #n_pt=[pt[0],d_gr ,n_seq]
            n_pt= [pt_sk,pt_lbl,d_gr ,n_seq]
            fset.append(n_pt)
        
        print (s,len(fset))        
              
        fname='/data/projects/py_ehr_2/Data/h_'+str(h)+'cl2_combined_tp.'+s
        pickle.dump(fset, open(fname, 'wb'), -1)

test 8956
train 29504
valid 4269


In [2]:
for s in ['test','train','valid']:    
    fname='/data/projects/py_ehr_2/Data/h_143cl2_combined_tp.'+s
    fset1=pickle.load(open(fname, 'rb'),encoding='bytes')
    print (s,len(fset1),fset1[0])

test 8956 [310327, 1, [63, 62, 61], [[[0], 1456], [[9], 124, 63, 62, 375, 239], [[32], 239, 62, 63], [[29], 239, 62, 63], [[189], 722, 63, 62, 129, 371, 399, 230], [[16], 62, 63, 230], [[98], 62, 239, 63, 449], [[28], 793, 230, 62, 124, 63, 239, 449], [[18], 239, 63, 62], [[28], 239, 62, 63], [[35], 239, 63, 62], [[28], 239, 62, 63], [[28], 63, 239, 62], [[28], 62, 239, 63], [[17], 230, 449, 59, 945, 124, 62, 63], [[12], 62, 63, 239], [[28], 63, 239, 62]]]
train 29504 [310048, 1, [63, 84, 231], [[[0], 1416], [[15], 57, 457, 84, 63], [[13], 63, 1416, 2504, 4515, 84, 925], [[21], 518, 57, 232, 84, 150, 63], [[43], 457, 132, 556, 57, 84, 63], [[8], 2669, 8, 556, 2375, 84, 384, 421, 1788, 63, 6], [[40], 68, 63, 1416, 84], [[15], 449, 57, 84, 457, 556, 63], [[28], 84, 559, 63, 675, 57], [[48], 1416, 63, 84], [[15], 388, 457, 57, 63, 84, 449], [[5], 63, 84, 268], [[19], 195, 457, 343, 1182, 53, 173, 553, 207, 84, 751, 34, 12, 31, 409, 212, 16, 158, 22, 80, 1259, 6, 135, 956, 189, 657, 56, 47

In [ ]:
emb_types = pickle.load(open('/data/projects/cerner_dev/cl2_hosp_pdata/h50_cl2_all.types.train', 'rb'),encoding='bytes')

for x in emb_types:
    if ('a_' in str(x)) or ( 'g_' in str(x))or ('r_' in str(x)) : print (x,emb_types[x])

In [ ]:
'''
pickle.dump(train_sl, open('hf50_bal_train', 'wb'), -1)
pickle.dump(test_sl, open('hf50_bal_test', 'wb'), -1)
pickle.dump(valid_sl, open('hf50_bal_valid', 'wb'), -1)

In [10]:
## Adding time and pt_sk to Readmission Data
for h in [143]: #[104 ,118 ,131 ,154 ,212 ,225 ,384 ,85 ,14 ,143]:
    t_file= '/data/projects/Readm/pdata/readm_diag_enc_'+ str(h)
    t_seqfile= t_file +'.visits.'
    t_labfile= t_file +'.labels.'
    t_timefile= t_file +'.days.'
    t_ptfile= t_file +'.pts.'
    
    for s in ['all']:#['test','train','valid']:
        fset=[]
        t_seqfile_s=t_seqfile+s
        t_labfile_s=t_labfile+s
        t_timefile_s= t_timefile +s
        t_ptfile_s= t_ptfile +s
        set_x = pickle.load(open(t_seqfile_s, 'rb'),encoding= 'bytes')
        set_y = pickle.load(open(t_labfile_s, 'rb'),encoding= 'bytes')
        set_t = pickle.load(open(t_timefile_s, 'rb'),encoding= 'bytes')
        set_p = pickle.load(open(t_ptfile_s, 'rb'),encoding= 'bytes')
        for pt_idx in range(len(set_y)):
            pt_sk= set_p[pt_idx]
            pt_lbl= set_y[pt_idx]
            pt_vis= set_x[pt_idx]
            pt_td= set_t[pt_idx]
            d_gr=[]
            n_seq=[]
            d_a_v=[]
            for v in range(len(pt_vis)):
                    nv=[]
                    nv.append([pt_td[v]])
                    nv.append(pt_vis[v])                   
                    n_seq.append(nv)
            n_pt= [pt_sk,pt_lbl,n_seq]
            fset.append(n_pt)
        
        print (s,len(fset))        
              
        fname='/data/projects/py_ehr_2/Data/Readm_h_'+str(h)+'cl2_combined_tp.'+s
        pickle.dump(fset, open(fname, 'wb'), -1)

all 18039


In [11]:
fset[1:3]

[[310046,
  0,
  [[[0], [54, 55, 8, 56, 4, 57, 58, 59, 60, 61, 62, 17, 63, 13, 36, 11]],
   [[99], [61, 56, 4, 57, 60, 58, 59, 55, 36, 62, 17, 63, 13, 11, 8, 54]],
   [[8], [61, 56, 4, 57, 58, 59, 55, 60, 62, 63, 13, 54, 36, 11, 8, 64]],
   [[18], [62, 4, 57, 5, 58, 59, 55, 61, 56, 65, 66, 13, 36, 11, 8, 54, 63]],
   [[7], [61, 56, 4, 57, 67, 58, 59, 55, 62, 68, 63, 13, 36, 11, 8, 54]],
   [[2], [61, 56, 4, 57, 67, 59, 55, 58, 62, 36, 8, 11, 68, 13, 63, 54]],
   [[116], [61, 56, 4, 57, 67, 58, 55, 59, 62, 68, 63, 13, 36, 11, 69, 54]],
   [[1], [61, 56, 4, 57, 67, 58, 59, 55, 11, 62, 68, 63, 13, 36, 69, 54]],
   [[1], [61, 56, 4, 57, 5, 58, 59, 55, 62, 68, 13, 36, 11, 69, 54, 63]],
   [[40], [55, 56, 4, 57, 60, 58, 59, 61, 62, 64, 63, 13, 36, 11, 69, 54]],
   [[210], [61, 56, 4, 57, 67, 58, 59, 55, 54, 62, 68, 63, 13, 36, 11, 69]],
   [[140], [61, 56, 4, 57, 67, 59, 55, 58, 62, 63, 54, 13, 36, 11, 69, 68]],
   [[147], [62, 4, 57, 58, 59, 55, 61, 56, 70, 63, 71, 13, 36, 11, 69, 54]],
   

In [8]:
emb_types = pickle.load(open('/data/projects/Readm/pdata/readm_diag_enc_143.types.all', 'rb'),encoding='bytes')

for x in emb_types:
    if ('Gender' in str(x) or  'Race' in str(x)): print (x,emb_types[x])

b'Gender_Male' 59
b'Gender_Not Mapped' 528
b'Race_Not Mapped' 527
b'Race_Pacific Islander' 1200
b'Race_Asian' 1201
b'Gender_Unknown/Invalid' 11800
b'Gender_Female' 7
b'Race_Caucasian' 3
b'Race_Hispanic' 2252
b'Race_African American' 56
b'Race_Other' 1490
b'Race_Unknown' 2451
b'Race_Native American' 1606


In [3]:
## Adding time and pt_sk to HF Data

for h in [143]: #[104 ,118 ,131 ,154 ,212 ,225 ,384 ,85 ,14 ,143]:
    t_file= '/data/projects/cerner_dev/cl2_hosp_pdata/h50_cl2_all_hosp_'+ str(h)
    t_seqfile= t_file +'.visits.'
    t_labfile= t_file +'.labels.'
    t_timefile= t_file +'.days.'
    t_ptfile= t_file +'.pts.'
    
    for s in ['test','train','valid']:
        fset=[]
        t_seqfile_s=t_seqfile+s
        t_labfile_s=t_labfile+s
        t_timefile_s= t_timefile +s
        t_ptfile_s= t_ptfile +s
        set_x = pickle.load(open(t_seqfile_s, 'rb'),encoding= 'bytes')
        set_y = pickle.load(open(t_labfile_s, 'rb'),encoding= 'bytes')
        set_t = pickle.load(open(t_timefile_s, 'rb'),encoding= 'bytes')
        set_p = pickle.load(open(t_ptfile_s, 'rb'),encoding= 'bytes')
        for pt_idx in range(len(set_y)):
            pt_sk= set_p[pt_idx]
            pt_lbl= set_y[pt_idx]
            pt_vis= set_x[pt_idx]
            pt_td= set_t[pt_idx]
            d_gr=[]
            n_seq=[]
            d_a_v=[]
            for v in range(len(pt_vis)):
                    nv=[]
                    nv.append([pt_td[v]])
                    nv.append(pt_vis[v])                   
                    n_seq.append(nv)
            n_pt= [pt_sk,pt_lbl,n_seq]
            fset.append(n_pt)
      
        print (s,len(fset))        
              
        fname='/data/projects/py_ehr_2/Data/hf50_cl2_h'+str(h)+'_ref_t.'+s
        pickle.dump(fset, open(fname, 'wb'), -1)


test 8956
train 29504
valid 4269


In [4]:
fset[1:3]

[[12788042,
  0,
  [[[0], [63, 84, 149, 6, 8, 26]],
   [[103],
    [1479, 63, 84, 149, 1181, 172, 12, 102, 8, 209, 331, 167, 135, 6, 4, 93]],
   [[4], [149, 93, 135, 172, 1181, 63, 84]],
   [[1], [63, 84, 30, 149, 49, 12]],
   [[207], [84, 8, 149, 4, 63, 6, 26]]]],
 [12977320, 0, [[[0], [6, 8, 149, 84, 63]]]]]

In [5]:
train_sl= pickle.load(open('/data/projects/py_ehr_2/Data/hf50_cl2_h143_ref_t.train', 'rb'), encoding='bytes')
test_sl= pickle.load(open('/data/projects/py_ehr_2/Data/hf50_cl2_h143_ref_t.test', 'rb'), encoding='bytes')
valid_sl= pickle.load(open('/data/projects/py_ehr_2/Data/hf50_cl2_h143_ref_t.valid', 'rb'), encoding='bytes')

In [6]:
print (len(train_sl),len(valid_sl),len(test_sl))

29504 4269 8956


In [17]:
### Reparse the multilist [diag][medication][others] Readm

for h in [143]: #[104 ,118 ,131 ,154 ,212 ,225 ,384 ,85 ,14 ,143]:
    t_file= '/data/projects/Readm/pdata/readm_mde_'+ str(h)+'_s'
    t_seqfile= t_file +'.visits.'
    t_seqfileS= t_file +'.visits_spl.'
    t_labfile= t_file +'.labels.'
    t_timefile= t_file +'.days.'
    t_ptfile= t_file +'.pts.'
    
    for s in ['all']:#['test','train','valid']:
        fset=[]
        fsetS=[]
        t_seqfile_s=t_seqfile+s
        t_seqfileS_s=t_seqfileS+s
        t_labfile_s=t_labfile+s
        t_timefile_s= t_timefile +s
        t_ptfile_s= t_ptfile +s
        set_x = pickle.load(open(t_seqfile_s, 'rb'),encoding= 'bytes')
        set_xs = pickle.load(open(t_seqfileS_s, 'rb'),encoding= 'bytes')
        set_y = pickle.load(open(t_labfile_s, 'rb'),encoding= 'bytes')
        set_t = pickle.load(open(t_timefile_s, 'rb'),encoding= 'bytes')
        set_p = pickle.load(open(t_ptfile_s, 'rb'),encoding= 'bytes')
        for pt_idx in range(len(set_y)):
            pt_sk= set_p[pt_idx]
            pt_lbl= set_y[pt_idx]
            pt_vis= set_x[pt_idx]
            pt_visS= set_xs[pt_idx]
            pt_td= set_t[pt_idx]
            d_gr=[]
            n_seq=[]
            n_seqS=[]
            d_a_v=[]
            for v in range(len(pt_vis)):
                    nv=[]
                    nv.append([pt_td[v]])
                    nv.append(pt_vis[v])                   
                    n_seq.append(nv)
            n_pt= [pt_sk,pt_lbl,n_seq]
            fset.append(n_pt)
            for vs in range(len(pt_visS)):
                    nvs=[]
                    nvs.append([pt_td[vs]])## time First
                    nvs.append(pt_visS[vs])                   
                    n_seqS.append(nvs)
            n_ptS= [pt_sk,pt_lbl,n_seqS]
            fsetS.append(n_ptS)
        
        #print (s,len(fset),fset[0],len(fsetS),fsetS[0])        
              
        fname='/data/projects/py_ehr_2/Data/Readm_h_'+str(h)+'cl2_mde_tp.'+s
        fnameS='/data/projects/py_ehr_2/Data/Readm_h_'+str(h)+'cl2_mde_tp_S.'+s

        pickle.dump(fset, open(fname, 'wb'), -1)
        pickle.dump(fsetS, open(fnameS, 'wb'), -1)

In [14]:
print (s,len(fset),fset[0][2][0],len(fsetS),fsetS[0][2][0])    

all 18039 [[0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]] 18039 [[0], [[], [], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]]


In [18]:
### Split Readm data to subset

dataSize = len(fsetS)
nTest = int(0.2 * dataSize)
nValid = int(0.1 * dataSize)
    
random.shuffle(fsetS)
random.shuffle(fset)

test_set = fset[:nTest]
valid_set = fset[nTest:nTest+nValid]
train_set = fset[nTest+nValid:]

pickle.dump(test_set, open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp.test', 'wb'), -1)
pickle.dump(train_set, open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp.train', 'wb'), -1)
pickle.dump(valid_set, open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp.valid', 'wb'), -1)

test_set1 = fsetS[:nTest]
valid_set1 = fsetS[nTest:nTest+nValid]
train_set1 = fsetS[nTest+nValid:]

pickle.dump(test_set1, open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp_S.test', 'wb'), -1)
pickle.dump(train_set1, open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp_S.train', 'wb'), -1)
pickle.dump(valid_set1, open('/data/projects/py_ehr_2/Data/Readm_h143_cl2_mde_tp_S.valid', 'wb'), -1)


In [12]:
### Reparse the multilist [diag][medication][others] HF

for h in [143]: #[104 ,118 ,131 ,154 ,212 ,225 ,384 ,85 ,14 ,143]:
    t_file= '/data/projects/cerner_dev/cl2_hosp_pdata/hf50_h'+ str(h)+'_all_cl2_spl'
    t_seqfile= t_file +'.visits.'
    t_seqfileS= t_file +'.visits_spl.'
    t_labfile= t_file +'.labels.'
    t_timefile= t_file +'.days.'
    t_ptfile= t_file +'.pts.'
    
    for s in ['test','train','valid']:
        fset=[]
        fsetS=[]
        t_seqfile_s=t_seqfile+s
        t_seqfileS_s=t_seqfileS+s
        t_labfile_s=t_labfile+s
        t_timefile_s= t_timefile +s
        t_ptfile_s= t_ptfile +s
        set_x = pickle.load(open(t_seqfile_s, 'rb'),encoding= 'bytes')
        set_xs = pickle.load(open(t_seqfileS_s, 'rb'),encoding= 'bytes')
        set_y = pickle.load(open(t_labfile_s, 'rb'),encoding= 'bytes')
        set_t = pickle.load(open(t_timefile_s, 'rb'),encoding= 'bytes')
        set_p = pickle.load(open(t_ptfile_s, 'rb'),encoding= 'bytes')
        for pt_idx in range(len(set_y)):
            pt_sk= set_p[pt_idx]
            pt_lbl= set_y[pt_idx]
            pt_vis= set_x[pt_idx]
            pt_visS= set_xs[pt_idx]
            pt_td= set_t[pt_idx]
            d_gr=[]
            n_seq=[]
            n_seqS=[]
            d_a_v=[]
            for v in range(len(pt_vis)):
                    nv=[]
                    nv.append([pt_td[v]])
                    nv.append(pt_vis[v])                   
                    n_seq.append(nv)
            n_pt= [pt_sk,pt_lbl,n_seq]
            fset.append(n_pt)
            for vs in range(len(pt_visS)):
                    nvs=[]
                    nvs.append([pt_td[vs]])## time First
                    nvs.append(pt_visS[vs])                   
                    n_seqS.append(nvs)
            n_ptS= [pt_sk,pt_lbl,n_seqS]
            fsetS.append(n_ptS)
        
        #print (s,len(fset),fset[0],len(fsetS),fsetS[0])        
              
        fname='/data/projects/py_ehr_2/Data/hf50_cl2_h'+str(h)+'_ref_t_S.'+s
        pickle.dump(fsetS, open(fname, 'wb'), -1)